# Task 5
#### ID1: 319206850
#### ID2: 203614094

## Part 1 - Bayesian Approach
#### recap
our test question from task 2 was whether the numeric variable $tenure$ is different between the categories that defined by the binary variable $churn$.

In [3]:
import pandas as pd
import numpy as np
import random

### Question 1 - sampling

In [4]:
random.seed(319206850)
orig_data = pd.read_csv('telco_data.csv')

orig_data.loc[orig_data.Churn == 'Yes', 'Churn'] = 1
orig_data.loc[orig_data.Churn == 'No', 'Churn'] = 0

obs_data = orig_data.sample(200)
past_data = orig_data.loc[~orig_data.index.isin(obs_data.index)].sample(n=1000)



### Question 2 - Dichotomation

In [5]:
X = obs_data['tenure'].values
t = sorted(X)[int(len(X)/2)]
print(f"tau = {t}")
Z = [1 if x > t else 0 for x in X]
obs_data['Z'] = Z
print(obs_data[['Churn', 'Z']])

tau = 29
     Churn  Z
3534     1  1
1622     0  1
4347     0  1
4203     0  0
6419     0  1
...    ... ..
1068     1  1
5279     0  1
4033     0  1
4080     0  0
6776     0  1

[200 rows x 2 columns]


### Question 3 - Estimation

#### 3.a - Bootstrap

In [6]:
from math import log2, sqrt
z_a = 1.96
B = 400

In [7]:
def ci_bootstrap_se(X, Y, orig_b, bootstap_b):
    se = []
    for i in range(len(bootstap_b[0])):
        se.append(sqrt((sum([b[i]**2 for b in bootstap_b]))/B - (sum([b[i] for b in bootstap_b])/B)**2))
    ci = [[orig_b[i] - z_a*se[i], orig_b[i] + z_a*se[i]] for i in range(len(orig_b))]
    print_ci(ci, "bootstrap normal approx.")
    return ci

In [8]:
def calc_y_ci_bootstrap(data, orig_psi):
    bootstrap_psi = []
    n = len(data.index)
    b = 0
    while b < B:
        indexes = random.choices(list(range(n)), k=n)
        sampled_data = data.iloc[indexes]
        n0 = len(sampled_data[sampled_data['Churn'] == 0].index)
        n1 = len(sampled_data[sampled_data['Churn'] == 1].index)
        e_p0 = len(sampled_data[(sampled_data['Z'] == 1) & (sampled_data['Churn'] == 0)].index) / n0  
        e_p1 = len(sampled_data[(sampled_data['Z'] == 1) & (sampled_data['Churn'] == 1)].index) / n1
        if e_p1 == 0:
            continue
        bootstrap_psi.append(log2(e_p0 / e_p1))
        b += 1
    se = sqrt((sum([x**2 for x in bootstrap_psi]))/B - (sum([y for y in bootstrap_psi])/B)**2)
    ci = [orig_psi - z_a*se, orig_psi + z_a*se]
    return ci

In [9]:
obs_data = pd.read_csv('telco_data.csv')
N0, N1 = len(obs_data[obs_data['Churn'] == 0].index), len(obs_data[obs_data['Churn'] == 1].index)
S0 = len(obs_data[(obs_data['Z'] == 1) & (obs_data['Churn'] == 0)].index)
S1 = len(obs_data[(obs_data['Z'] == 1) & (obs_data['Churn'] == 1)].index)
e_p0 = S0 / N0
e_p1 = S1 / N1
psi = round(log2(e_p0 / e_p1), 3)
print(f"etimate for psi: {psi}")
bootstrap_ci = calc_y_ci_bootstrap(obs_data, psi)
print(f"bootstrap normal approx. ci: {bootstrap_ci}")

KeyError: 'Z'

#### 3.b - Uniform Prior

In [10]:
uni_e_p0 = (S0 + 1) / (N0 + 2)
uni_e_p1 = (S1 + 1) / (N1 + 2)
uni_e_psi = log2(uni_e_p0 / uni_e_p1)
print(f"uniform prior estimator for psi: {round(uni_e_psi, 3)}")

NameError: name 'S0' is not defined

## Part 2 - Missing Data

### Question 1,2 - Data sampling and Linear Regression

In [11]:
def adjust_data(df, cat):
    df = df[cat]
    df = df[pd.to_numeric(df['TotalCharges'], errors='coerce').notnull()]
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    df = df.sample(1000)
    df['gender'].loc[(df['gender'] == 'Male')] = 1
    df['gender'].loc[(df['gender'] == 'Female')] = 0
    df['TotalCharges'] = [float(x) for x in df['TotalCharges'].values]
    df['tenure'] = [float(x) for x in df['tenure'].values]
    return df

In [12]:
def calc_b(X, Y, b_only=False):
    Xt = X.transpose()
    XtY = Xt.dot(Y)
    XtX = Xt.dot(X)
    inv_matrix = np.linalg.pinv(XtX)
    b = inv_matrix.dot(XtY)
    if b_only:
        return b
    return b, inv_matrix

In [13]:
def get_b_inv(df, b_only=True):
    X = df.drop(['TotalCharges'], axis=1)
    X.insert(0, 'ones', 1)
    X = X.to_numpy(dtype='float')
    Y = df['TotalCharges'].to_numpy(dtype='float')
    return calc_b(X, Y, b_only)

In [14]:
def ci_matrix(b, inv_matrix, n):
    ci = []
    df_res = n - len(b)
    for i in range(len(inv_matrix)):
        se = (df_res*inv_matrix[i][i])**0.5
        ci.append([b[i] - z_a*se, b[i] + z_a*se])
    return ci

In [45]:
random.seed(319206850)

cat = ['gender', 'SeniorCitizen', 'tenure', 'TotalCharges']

orig_data = pd.read_csv('telco_data.csv')
sampled_seen = adjust_data(orig_data, cat)
b, inv_mat = get_b_inv(sampled_seen, b_only=False)
b_ci = ci_matrix(b, inv_mat, len(sampled_seen.index))
print_b_ci(b, b_ci, "standard linear regression")

standard linear regression:
	estimator for b0 is -257.171	 and the ci is: [-260.983, -253.359]
	estimator for b1 is -16.307	 and the ci is: [-20.225, -12.39]
	estimator for b2 is 336.614	 and the ci is: [331.564, 341.663]
	estimator for b3 is 79.805	 and the ci is: [79.725, 79.885]



### Question 3 - Erasing

In [46]:
sampled_unseen = sampled_seen.sort_values(by=['TotalCharges'])
erased_Y = []
for i, y in enumerate(sampled_unseen['TotalCharges'].values):
    p_i = 0.1 + (i / 1250)
    r = random.random()
    if r > p_i:
        erased_Y.append(y)
    else:
        erased_Y.append(np.nan)
sampled_unseen['TotalCharges'] = erased_Y
print(f"number of Y erased: {sampled_unseen['TotalCharges'].isna().sum()}")

number of Y erased: 484


### Question 4 

In [47]:
def print_b_ci(b, ci, title):
    print(f"{title}:")
    for i, e in enumerate(b):
        if ci is None:
            print(f"\testimator for b{i} is {round(e, 3)}")
        else:
            print(f"\testimator for b{i} is {round(e, 3)}\t and the ci is: {[round(x, 3) for x in ci[i]]}")
    print()

#### 4.a - Linear Regression with Complete Data

In [48]:
data_a = sampled_unseen.dropna()
b_a, inv_a = get_b_inv(data_a, b_only=False)
ci_a = ci_matrix(b_a, inv_a, len(data_a.index))
print_b_ci(b_a, ci_a, "4.a - linear regression with complete data")

4.a - linear regression with complete data:
	estimator for b0 is -175.9	 and the ci is: [-179.355, -172.445]
	estimator for b1 is 113.463	 and the ci is: [109.556, 117.37]
	estimator for b2 is 416.282	 and the ci is: [411.176, 421.388]
	estimator for b3 is 63.536	 and the ci is: [63.444, 63.627]



#### 4.b - Regression Imputation

In [49]:
def reg_imp(df):
    R = [0 if np.isnan(x) else 1 for x in df['TotalCharges'].values]
    X = df.drop(['TotalCharges'], axis=1)
    X.insert(0, 'ones', 1)
    X = X.to_numpy(dtype='float')
    Y = df['TotalCharges'].to_numpy(dtype='float')
    XtX = np.zeros((len(X[0]), len(X[0])))
    XtY = np.zeros((len(X[0]), 1))
    for i in range(len(R)):
        XtX += R[i] * (X[i].reshape(len(X[i]), 1)).dot(X[i].reshape(1, len(X[i])))
        XtY += (X[i].reshape(len(X[i]), 1)) * Y[i] if R[i] == 1 else np.zeros((len(X[0]), 1))
    inv = np.linalg.pinv(XtX)
    b = inv.dot(XtY).reshape(1, len(X[0]))[0]
    return b, inv

In [50]:
def compare_CIs(ci_1, ci_2):
    print(f"comparing CIs: {ci_1['name']} vs {ci_2['name']}:")
    ratio = []
    for i in range(len(ci_1['data'])):
        d1, d2 = ci_1['data'][i][1] - ci_1['data'][i][0], ci_2['data'][i][1] - ci_2['data'][i][0] 
        ratio.append(d1 / d2)
    avg_ratio = sum(ratio) / len(ratio)
    if avg_ratio < 1:
        smaller = ci_1['name']
    else:
        smaller = ci_2['name']
        avg_ratio = 1 / avg_ratio
    print(f"CIs of {smaller} are {round(100*(1-avg_ratio), 2)}% smaller on average")

In [51]:
b_b, inv_b = reg_imp(sampled_unseen)
ci_b = ci_matrix(b_b, inv_b, len(sampled_unseen.index))
print_b_ci(b_b, ci_b, "4.b - regression imputation")
compare_CIs({'name': 'regression imputation method', 'data': ci_b}, {'name': 'complete data method', 'data': ci_a})

4.b - regression imputation:
	estimator for b0 is -175.9	 and the ci is: [-180.719, -171.081]
	estimator for b1 is 113.463	 and the ci is: [108.014, 118.912]
	estimator for b2 is 416.282	 and the ci is: [409.161, 423.404]
	estimator for b3 is 63.536	 and the ci is: [63.408, 63.664]

comparing CIs: regression imputation method vs complete data method:
CIs of complete data method are 28.3% smaller on average


As we can see, the estimators are the same for 4.a and 4.b method. however, the CIs of 4.a method were 28.3% smaller than the CIs of 4.b method.

#### 4.c - Multiple Imputation

In [52]:
def multiple_imputation(df, b_b, M):
    X = df.drop('TotalCharges', axis= 1)
    X.insert(0, 'ones', 1)
    X = X.to_numpy(dtype='float')
    Y_pred = X.dot(b_b)
    Y_orig = df['TotalCharges'].values
    Y_full = [Y_pred[i] if np.isnan(Y_orig[i]) else Y_orig[i] for i in range(len(X))]
    ms_res = sum([(Y_full[i] - Y_pred[i])**2 for i in range(len(X))]) / (len(X) - len(X[0]))
    all_b, all_inv = [], []
    for m in range(M):
        Y_tmp = []
        for i in range(len(X)):
            mu = sum([X[i][j] * b_b[j] for j in range(len(b_b))])
            stdv = sqrt(ms_res)
            y = np.random.normal(mu, stdv) if np.isnan(Y_orig[i]) else Y_orig[i]       
            Y_tmp.append(y)
        tmp_data = df
        tmp_data['TotalCharges'] = Y_tmp
        b_tmp, inv = get_b_inv(tmp_data, b_only=False)
        all_inv.append(inv)
        all_b.append(b_tmp)
    return all_b, all_inv

In [53]:
M = 400
b_c, inv_c = multiple_imputation(sampled_unseen, b_b, M)
b_MI = [sum([b_c[i][j] for i in range(M)]) / M for j in range(len(b_b))]
print_b_ci(b_MI, None, "4.c - multiple imputation")

4.c - multiple imputation:
	estimator for b0 is -183.703
	estimator for b1 is 115.474
	estimator for b2 is 392.986
	estimator for b3 is 62.848



#### 4.d - rubin's formula

In [54]:
var_MI = [(sum(inv_c[i][j][j] for i in range(M)) / M) + 
          ((M+1) / M*(M-1))*sum([(b_c[i][j] - b_MI[j])**2 for i in range(M)]) for j in range(len(b_MI))]

ci_d = ci_matrix(b_MI, np.diag(var_MI), len(sampled_unseen.index))
print_b_ci(b_MI, ci_d, "4.d - MI + CIs of rubin's formula")

4.d - MI + CIs of rubin's formula:
	estimator for b0 is -183.703	 and the ci is: [-187.515, -179.891]
	estimator for b1 is 115.474	 and the ci is: [111.556, 119.391]
	estimator for b2 is 392.986	 and the ci is: [387.937, 398.036]
	estimator for b3 is 62.848	 and the ci is: [62.769, 62.928]



#### 4.e - logistic regression

In [26]:
from statsmodels.discrete.discrete_model import Logit

ModuleNotFoundError: No module named 'statsmodels'